In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os

%matplotlib inline

# ETMT Data Analysis

### Load in the ETMT data

Load in the columns from the datafile. 

*Note, start from zero.*

For data collected from Diamond Light Source in 2022 the format for the columns is given as. It is advised that this template is used in future;

- A(0) Total Time (s)
- B(1) Cycle Elapsed Time (s)
- C(2) Total Cycles
- D(3) Elapsed Cycles
- E(4) Step
- F(5) Total Cycle Count(Load Frame Waveform)
- G(6) Total Cycle Count(Temperature Waveform)
- H(7) Time Stamp(PC)
- I(8) Position(Load Frame:Linear Position Encoder) (mm)
- J(9) Displacement(Load Frame:Linear Position Encoder) (mm)
- K(10) Strain(Load Frame:Linear Position Encoder) (%)
- L(11) Force(Load Frame:Load) (N)
- M(12) Stress(Load Frame:Load) (MPa)
- N(13) Strain(Load Frame:Strain 1) (%)
- O(14) User-Defined(Temperature:Temperature) (C)
- P(15) Temperature(Temperature Controller 1:Temperature) (C)
- Q(16) User-Defined(Temperature:Current) (Amps)
- R(17) User-Defined(Temperature:Voltage) (Volts)
- S(18) Drive(Load Frame Waveform) (%)
- T(19) Command(Load Frame Waveform) (%)
- U(20) Error(Load Frame Waveform) (%)
- V(21) Command(Temperature Waveform) (%)
- W(22) Drive(Temperature Waveform) (%)
- X(23) Error(Temperature Waveform) (%)
- Y(24) Command(Temperature Controller 1 Waveform) (C)
- Z(25) Drive(Temperature Controller 1 Waveform) (%)

The main **inputs** are therefore:

- A(0) Time (s)
- I(8) Position/Displacement (mm)
- L(11) Load (N)
- 0(14) ETMT Temperature (C)
- P(15) Eurotherm Temperature (C)
- Q(16) Current (Amps)
- R(17) Voltage (V)

And we will need the command frame signal to determine the start and end of deformation:

- T(19) Command Frame Signal (%)

The main **ouptuts** will then be: 

- 0 = Time (s)
- 1 = Position/Displacement (mm)
- 2 = Load (N)
- 3 = ETMT Temperature (C)
- 4 = Eurotherm Temperature (C)
- 5 = Current (Amps)
- 6 = Voltage (V)
- 7 = Command Frame Signal (%)

In [ ]:
use_columns = (0,8,11,14,15,16,17,19)

time = 0
position = 1
load = 2
etmt_temperature = 3
eurotherm_temperature = 4
current = 5
voltage = 6
frame = 7

In [ ]:
experiment_number = "039"
experiment_name = "039_Ti64_TIFUN-T4_TD_Deform_820C_0p1mms-"

input_file = f"../../SXRD_raw_data/diamond_2022/rawdata_ETMT/{experiment_name}/Test1/Test1.steps.tracking.csv"
output_folder = f"../../SXRD_analysis/diamond_2022/{experiment_name}/ETMT-output"

ETMT_data = np.loadtxt(input_file, skiprows=1, delimiter=',', usecols = use_columns)
print(ETMT_data)

Find the start and end of the loading test using `Command Frame Signal`, when this is above a *very small* value. Check the ETMT data to see what the command frame value is at the start of deformation, but typically whole number percentages are applied right at the start of deformation.

In [ ]:
def above_value (array, number: int = 0.0005):
    output_array = np.zeros((2,1))
    for index in range (len(array)):
        if abs(array[index]) > abs (number):
            value = array[index]
            element = index
            output_array = np.append(output_array, [[value], [element]], axis=1)
    return output_array

def closest_value (array, number):
    value = array[0]
    for index in range (len(array)):
        if abs (number - array[index]) < abs (number - value):
            value = array[index]
            element = index
    return value, element

def sequence_checker(array, sequence_length: int = 4):
    """ Check for sequence of consecutive numbers in array
    and return array when sequence has been found.
    """
    new_array = []
    for i in range(len(array) - 1):
        if(array[i] == array[i + 1]):
            new_array += [array[i]]
            if(len(new_array) == sequence_length):
                break
        else:
            new_array = []
    return new_array

In [ ]:
frame_signal = ETMT_data[:,frame]
frame_non_zero = above_value(frame_signal, 0.03)
start_value = frame_non_zero[0][1]
start_index = int(frame_non_zero[1][1])
end_value = frame_non_zero[0][-1]
end_index = int(frame_non_zero[1][-1])
print("The start of deformation is at index", start_index, ", with a Command Frame Signal value of", start_value, "%")
print("The end of deformation is at index", end_index, ", with a Command Frame Signal value of", end_value, "%")

If the sample did not go to zero load immediately after deformation, but was held in position control, then we can detect the `Command Frame Signal` for a hold in position, which will be a series of the same value during the hold period.

In [ ]:
# frame_position_hold = sequence_checker(ETMT_data[start_index:end_index,frame])
# end_value, end_index = closest_value(ETMT_data[:,frame],frame_position_hold[0])

# print("The end of deformation has been changed to index", start_index, ", with a Command Frame Signal value of", start_value, "%") 
# print("This is to remove a hold in position immediately after deformation.")

### Plotting out the ETMT data

Calculate the engineering stress from the load using the `gauge_area` of the sample. And calculate the engineering strain from the length change using the `gauge_length` of the sample.

In [ ]:
gauge_area = 8
gauge_length = 2

time_array = ETMT_data[start_index:end_index,time]
load_array = ETMT_data[start_index:end_index, load]
position_array = ETMT_data[start_index:end_index,position]
displacement_array = position_array - position_array[0]

stress_array = load_array / gauge_area
strain_array = displacement_array / gauge_length

etmt_temperature_array = ETMT_data[start_index:end_index, etmt_temperature]
eurotherm_temperature_array = ETMT_data[start_index:end_index, eurotherm_temperature]

current_array = ETMT_data[start_index:end_index, current]
voltage_array = ETMT_data[start_index:end_index, voltage]

Calculate the true stress and true strain of the sample using the **Resistance Method**. An explanation of this method can be found in this paper by [Roebuck, Cox and Reed](https://www.tms.org/Superalloys/10.7449/2004/Superalloys_2004_523_528.pdf)

Using the resistance method we can calculate true strain $\varepsilon_p = \ln{ \sqrt{ \frac{R_l}{R_s}}}$ where the resistance during the test, $R_l$ and at the start $R_s$ are calculated from the measured voltage and measured current $R=\frac{V}{I}$. In this case, the current is the amount delivered to the sample by the ETMT, and the voltage drop across the centre of the sample is measured by two resistivity wires (of the same material, typically either pure Pt, Cu, or Ni80Cr wire).

*Note, we have already found the start of the deformation part of the test using the `Command Frame Signal`, when it is non-zero, or above a very small value.*

In [ ]:
start_current = ETMT_data[start_index,current]
start_voltage = ETMT_data[start_index,voltage]

start_resistance = start_voltage / start_current
resistance_array = voltage_array / current_array

true_strain_array = np.log(np.sqrt(resistance_array / start_resistance))

engineering_strain_array = np.expm1(true_strain_array)

true_stress_array = stress_array*(1 + engineering_strain_array)

Plot out (and save) the figures showing the applied stress, change in temperature and true stress versus true strain response. Check these signals carefully as it is possible the resistivity wires could be noisey or introduce erroneous results.

In [ ]:
plt.rc('xtick',labelsize=16)
plt.rc('ytick',labelsize=16)
plt.rc('legend',fontsize=16)
plt.rc('axes',linewidth=1)
plt.rc('xtick.major',width=2,size=10)
plt.rc('xtick.minor', width=2, size=5)
plt.rc('ytick.major',width=2,size=10)
plt.rc('ytick.minor',width=2,size=5)

fig,((ax1,ax2),(ax3,ax4),(ax5,ax6))=plt.subplots(3, 2, figsize=(20,15))

ax1.plot(position_array, stress_array, marker = "o", color = "green", markersize = 5)
ax1.set_xlabel("Position (mm)", fontsize = 20)
ax1.set_ylabel("Applied Stress, ${\sigma_E}$ (MPa)", fontsize = 20)
ax1.minorticks_on()

ax2.plot(position_array, resistance_array, marker = "o", color = "green", markersize = 5)
ax2.set_xlabel("Position (mm)", fontsize = 20)
ax2.set_ylabel("Resistance, ${(\Omega)}$", fontsize = 20)
ax2.minorticks_on()

ax3.plot(position_array, etmt_temperature_array, marker = "o", color = "red", markersize = 5)
ax3.set_xlabel("Position (mm)", fontsize = 20)
ax3.set_ylabel("ETMT Control Temperature, ${^\circ C}$", fontsize = 20)
ax3.minorticks_on()

ax4.plot(position_array, eurotherm_temperature_array, marker = "o", color = "red", markersize = 5)
ax4.set_xlabel("Position (mm)", fontsize = 20)
ax4.set_ylabel("Eurotherm Temperature, ${^\circ C}$", fontsize = 20)
ax4.minorticks_on()

ax5.plot(strain_array, stress_array, marker = "o", color = "blue", markersize = 5)
ax5.set_ylabel("Applied Strain, ${\epsilon_E}$", fontsize = 20)
ax5.set_ylabel("Applied Stress, ${\sigma_E}$, (MPa)", fontsize = 20)
ax5.minorticks_on()

ax6.plot(true_strain_array, true_stress_array, marker = "o", color = "blue", markersize = 5)
ax6.set_xlabel("True Strain, ${\epsilon_T}$", fontsize = 20)
ax6.set_ylabel("True Stress, ${\sigma_T}$, (MPa)", fontsize = 20)
ax6.minorticks_on()

plt.tight_layout()

# check output folder exists
CHECK_FOLDER = os.path.isdir(output_folder)

if not CHECK_FOLDER:
    os.makedirs(output_folder)
    print("Created folder : ", output_folder)

plt.savefig("{output_folder}/ETMT_figures_{experiment_number}.png".format(output_folder = output_folder, experiment_number = experiment_number))

Write out an output ETMT datafile, which can be used for synicing the measured thermomechanical material response with the micromechanical elastic lattice strain response recorded using synchrotron X-ray diffraction (SXRD) pattern images.

The output ETMT datafile contains the following:

- 0 = Time (s)
- 1 = Load (N)
- 2 = Displacement (mm)
- 3 = Applied Stress (MPa)
- 4 = True Strain
- 5 = True Stress (MPa)
- 6 = ETMT Temperature (C)
- 7 = Eurotherm Temperature (C)
- 8 = Command Frame Signal (%)

In [ ]:
np.savetxt("{output_folder}/ETMT_output_{experiment_number}.txt".format(output_folder = output_folder, experiment_number = experiment_number), 
           np.c_[time_array, load_array, displacement_array, stress_array, true_strain_array, true_stress_array,
                etmt_temperature_array, eurotherm_temperature_array, ETMT_data[start_index:end_index,frame]], 
           header = "Time (s), Load (N), Displacement (mm), Applied Stress (MPa), True Strain, True Stress (MPa), ETMT Temperature (C), Eurotherm Temperature (C), Command Frame Signal (%)",
           delimiter = ',')